# 1. Import Dependencies

In [2]:
# Gym
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# helpers
import numpy as np 
import random
import os

# Import Stable_baseline stuff
from stable_baselines3 import PPO 
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces

In [14]:
Discrete(3).sample()

1

In [27]:
Box(0, 1, shape=(3,)).sample()

array([0.68393296, 0.6709114 , 0.9961112 ], dtype=float32)

In [32]:
Tuple((Discrete(3), Box(0, 1, shape=(3,)))).sample()

(1, array([0.28643075, 0.19812031, 0.5717966 ], dtype=float32))

In [33]:
Dict({'height': Discrete(2), 'speed': Box(0, 100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([60.943195], dtype=float32))])

In [43]:
MultiBinary(4).sample()

array([1, 1, 1, 1], dtype=int8)

In [56]:
MultiDiscrete([5, 2, 2]).sample()

array([4, 1, 1])

# 3. Building an Environment
- Build an agent to give us the best shower possible
- Randomly temperature
- 37 and 39 degree

In [66]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60
    def step(self, action):
        # Apply temp obj
        self.state += action-1

        # Decrease shower time
        self.shower_length -= 1

        # Calculate Reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        info = {} 
        return self.state, reward, done, info
    def rander(self):
        pass 
    def reset(self):
        self.state = np.array([38+random.randint(-3, 3)]).astype(float)
        self.shower_length = 60
        return self.state

In [67]:
env = ShowerEnv()

In [59]:
env.observation_space.sample()

array([43.49932], dtype=float32)

In [60]:
env.action_space.sample()

2

# 4. Test Environment

In [82]:
episodes = 5 
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    while not done:
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode: {}, Score: {}'.format(episode, score))
env.close()

Episode: 1, Score: 32
Episode: 2, Score: -46
Episode: 3, Score: 12
Episode: 4, Score: -30
Episode: 5, Score: 14


# 5. Train Model 

In [83]:
log_path = os.path.join('training', 'logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [84]:
model.learn(total_timesteps=100000)

Logging to training/logs/PPO_10
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -31.1    |
| time/              |          |
|    fps             | 662      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -31.6        |
| time/                   |              |
|    fps                  | 493          |
|    iterations           | 2            |
|    time_elapsed         | 8            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0065123737 |
|    clip_fraction        | 0.0177       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   

# 6. Save Model

In [86]:
shower_path = os.path.join('training', 'saved_models', 'PPO_Shower_Model')

In [87]:
model.save(shower_path)

In [88]:
del model

In [89]:
model = PPO.load(shower_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [90]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

/home/justin/github/rl_3hour_course/rl_3hours/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(-24.0, 54.99090833947008)